In [3]:
path_wrf_out = f'/home/sagus/Development/temp/wrfout_A_d01_2020-02-06_18:00:00'

In [206]:
from netCDF4 import Dataset
from osgeo import gdal_array, gdal, osr
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import cartopy.crs as crs
import xarray as xr
import glob
import numpy as np
from cartopy.feature import NaturalEarthFeature
import  geopandas as gdp
from wrf import (to_np, getvar, smooth2d, get_cartopy, cartopy_xlim,
                 cartopy_ylim, latlon_coords)

In [11]:
wrfout = Dataset(path_wrf_out)

In [226]:
wrfout

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    TITLE:  OUTPUT FROM WRF V4.1.1 MODEL
    START_DATE: 2020-02-06_18:00:00
    SIMULATION_START_DATE: 2020-02-06_18:00:00
    WEST-EAST_GRID_DIMENSION: 270
    SOUTH-NORTH_GRID_DIMENSION: 270
    BOTTOM-TOP_GRID_DIMENSION: 35
    DX: 4000.0
    DY: 4000.0
    AERCU_OPT: 0
    AERCU_FCT: 1.0
    IDEAL_CASE: 0
    DIFF_6TH_SLOPEOPT: 0
    AUTO_LEVELS_OPT: 2
    DIFF_6TH_THRESH: 0.1
    DZBOT: 50.0
    DZSTRETCH_S: 1.3
    DZSTRETCH_U: 1.1
    SKEBS_ON: 0
    SPEC_BDY_FINAL_MU: 1
    USE_Q_DIABATIC: 0
    GRIDTYPE: C
    DIFF_OPT: 1
    KM_OPT: 4
    DAMP_OPT: 3
    DAMPCOEF: 0.2
    KHDIF: 0.0
    KVDIF: 0.0
    MP_PHYSICS: 8
    RA_LW_PHYSICS: 1
    RA_SW_PHYSICS: 2
    SF_SFCLAY_PHYSICS: 2
    SF_SURFACE_PHYSICS: 2
    BL_PBL_PHYSICS: 2
    CU_PHYSICS: 0
    SF_LAKE_PHYSICS: 0
    SURFACE_INPUT_SOURCE: 1
    SST_UPDATE: 0
    GRID_FDDA: 0
    GFDDA_INTERVAL_M: 0
    GFDDA_END_H: 0
    GRID_SFDDA: 0


In [225]:
wrfout.dimensions['west_east']

<class 'netCDF4._netCDF4.Dimension'>: name = 'west_east', size = 269

In [238]:
import wrf
import os
import pandas as pd
import rasterio
from rasterstats import zonal_stats
from affine import Affine

In [243]:
WRF_EXTENT = [-538001.0623448786,
              -538000.0000000792,
              537998.9376551214,
              537999.9999999208]


CBA_EXTENT = [-68.91031,
              -37.408794,
              -56.489685,
              -27.518177]

KM_PER_DEGREE = 111.32
RESOLUTION = 4

In [231]:
def getGeoT(extent, nlines, ncols):
    # Compute resolution based on data dimension
    resx = (extent[2] - extent[0]) / ncols
    resy = (extent[3] - extent[1]) / nlines
    return [extent[0], resx, 0, extent[3], 0, -resy]

In [250]:
def guardar_tif(geoTransform: list, target_prj: str,
                arr: np.ndarray, out_path: str):
    nw_ds = rasterio.open(out_path, 'w', driver='GTiff',
                          height=arr.shape[0],
                          width=arr.shape[1],
                          count=1, dtype=str(arr.dtype),
                          crs=target_prj,
                          transform=Affine.from_gdal(*geoTransform))
    nw_ds.write(arr, 1)
    nw_ds.close()

In [258]:
def cambiar_projection(in_array: np.ndarray):
    """Convert Grid to New Projection.
        Parameters
        ----------
        in_array

    """
    # WRF Spatial Reference System
    source_prj = osr.SpatialReference()
    source_prj.ImportFromProj4('+proj=lcc +lat_0=-32.500008 +lon_0=-62.7 '
                               '+lat_1=-60 +lat_2=-30 +x_0=0 +y_0=0 +R=6370000'
                               ' +units=m +no_defs')
    # Lat/lon WSG84 Spatial Reference System
    target_prj = osr.SpatialReference()
    target_prj.ImportFromProj4('+proj=longlat +ellps=WGS84 '
                               '+datum=WGS84 +no_defs')

    # se configura la matriz destino
    sizex = int(((CBA_EXTENT[2] - CBA_EXTENT[0]) * KM_PER_DEGREE) / RESOLUTION)
    sizey = int(((CBA_EXTENT[3] - CBA_EXTENT[1]) * KM_PER_DEGREE) / RESOLUTION)

    out_array = np.ndarray(shape=(int(in_array.coords['Time'].count()),
                           sizey, sizex))

    for t in range(in_array.coords['Time'].size):
        # loar gdal array y se le asigna la projección y transofrmación
        raw = gdal_array.OpenArray(np.flipud(in_array[t].values))
        raw.SetProjection(source_prj.ExportToWkt())
        raw.SetGeoTransform(getGeoT(WRF_EXTENT,
                                    raw.RasterYSize,
                                    raw.RasterXSize))

        grid = gdal.GetDriverByName('MEM').Create("tmp_ras",
                                                  sizex, sizey, 1,
                                                  gdal.GDT_Float32)
        # Setup projection and geo-transformation
        grid.SetProjection(target_prj.ExportToWkt())
        grid.SetGeoTransform(getGeoT(CBA_EXTENT,
                                     grid.RasterYSize,
                                     grid.RasterXSize))

        # reprojectamos
        gdal.ReprojectImage(raw,
                            grid,
                            source_prj.ExportToWkt(),
                            target_prj.ExportToWkt(),
                            gdal.GRA_NearestNeighbour,
                            options=['NUM_THREADS=ALL_CPUS'])

        out_array[t] = grid.ReadAsArray()

    return out_array, grid.GetGeoTransform(), grid.GetProjection()

In [228]:
# WRF Spatial Reference System
source_prj = osr.SpatialReference()
source_prj.ImportFromProj4('+proj=lcc +lat_0=-32.500008 +lon_0=-62.7 +lat_1=-60 +lat_2=-30 +x_0=0 +y_0=0 +R=6370000 +units=m +no_defs')

0

In [ ]:
HGT = wrf.getvar(wrfout, 'HGT', timeidx=wrf.ALL_TIMES)

In [ ]:
LU = wrf.getvar(wrfout, 'LU_INDEX', timeidx=wrf.ALL_TIMES)

In [246]:
LU[0]

<xarray.DataArray 'LU_INDEX' (south_north: 269, west_east: 269)>
array([[ 7.,  7.,  7., ..., 17., 17., 17.],
       [ 7.,  7.,  7., ..., 17., 17., 17.],
       [ 7.,  7.,  7., ..., 17., 17., 17.],
       ...,
       [16., 16., 16., ..., 10., 10., 10.],
       [16.,  7., 16., ..., 10., 10.,  9.],
       [16., 16., 16., ...,  9., 10., 10.]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 -68.91031 -68.864136 ... -57.32242
    XLAT     (south_north, west_east) float32 -37.21706 ... -27.518177
    XTIME    float32 0.0
    Time     datetime64[ns] 2020-02-06T18:00:00
Dimensions without coordinates: south_north, west_east
Attributes:
    FieldType:    104
    MemoryOrder:  XY 
    description:  LAND USE CATEGORY
    units:        
    stagger:      
    coordinates:  XLONG XLAT XTIME
    projection:   LambertConformal(stand_lon=-62.70000076293945, moad_cen_lat...

In [209]:
raw = gdal_array.OpenArray(LU[0].values)

In [232]:
raw.SetProjection(source_prj.ExportToWkt())
raw.SetGeoTransform(getGeoT(WRF_EXTENT, raw.RasterYSize, raw.RasterXSize))

0

In [239]:
guardar_tif(raw.GetGeoTransform(), raw.GetProjection(), raw.ReadAsArray(), 'lu_index.tif')

In [261]:
var_proj, geoTransform, target_prj = cambiar_projection(HGT)

In [262]:
guardar_tif(geoTransform, target_prj, var_proj[0], 'hgt_index_WGS84_cu.tif')

In [152]:
wrfout.dimensions['Time'].size

58

In [127]:
for date in T2.coords['Time'].values:
    print(date.d)

IndexError: invalid index to scalar variable.

In [155]:
str(T2.coords['Time'].values[56])[:16]

'2020-02-09T02:00'

In [151]:
str(a)[:16]

'2020-02-06T18:00'

In [161]:
path = '/home/sagus/Development/wrf-meteo/geotiff/2020_02/06/CBA_A_18_T2'

In [164]:
#lista = glob.glob('/home/sagus/Development/wrf-meteo/geotiff/2020_02/06/CBA_A_18_T2*')
lista = sorted(glob.glob(f'{path}*'), key=os.path.getmtime)

In [184]:
for item in lista:
    print(item[65:])

2020-02-06T18:00
2020-02-06T19:00
2020-02-06T20:00
2020-02-06T21:00
2020-02-06T22:00
2020-02-06T23:00
2020-02-07T00:00
2020-02-07T01:00
2020-02-07T02:00
2020-02-07T03:00
2020-02-07T04:00
2020-02-07T05:00
2020-02-07T07:00
2020-02-07T06:00
2020-02-07T08:00
2020-02-07T10:00
2020-02-07T09:00
2020-02-07T11:00
2020-02-07T13:00
2020-02-07T12:00
2020-02-07T14:00
2020-02-07T15:00
2020-02-07T16:00
2020-02-07T17:00
2020-02-07T18:00
2020-02-07T19:00
2020-02-07T20:00
2020-02-07T21:00
2020-02-07T22:00
2020-02-07T23:00
2020-02-08T00:00
2020-02-08T02:00
2020-02-08T01:00
2020-02-08T03:00
2020-02-08T04:00
2020-02-08T06:00
2020-02-08T05:00
2020-02-08T07:00
2020-02-08T08:00
2020-02-08T09:00
2020-02-08T10:00
2020-02-08T11:00
2020-02-08T12:00
2020-02-08T13:00
2020-02-08T15:00
2020-02-08T14:00
2020-02-08T16:00
2020-02-08T18:00
2020-02-08T17:00
2020-02-08T19:00
2020-02-08T20:00
2020-02-08T21:00
2020-02-08T22:00
2020-02-08T23:00
2020-02-09T00:00
2020-02-09T01:00
2020-02-09T02:00
2020-02-09T03:00


In [176]:
SHAPE_ZONAS = ('/home/sagus/Development/temp/shapefiles/'
                'Epec/Zona_A_ET_Corbertura_20200922.shp')

In [177]:
zonas_gdf = gdp.read_file(SHAPE_ZONAS)

In [192]:
tmp_gdf = zonas_gdf[['Name', 'geometry']]

In [193]:
tmp_gdf

,Name,geometry
0,Oeste,"POLYGON Z ((-64.21238 -31.42046 0.00000, -64.2..."
1,Suroeste,"POLYGON Z ((-64.22738 -31.43135 0.00000, -64.2..."
2,Este,"POLYGON Z ((-64.10740 -31.44216 0.00000, -64.1..."
3,Tablada,"POLYGON Z ((-64.19650 -31.39692 0.00000, -64.1..."
4,Jardin,"POLYGON Z ((-64.17663 -31.44654 0.00000, -64.1..."
5,Rodriguez del Busto,"POLYGON Z ((-64.23433 -31.33951 0.00000, -64.2..."
6,Don Bosco,"POLYGON Z ((-64.27736 -31.43433 0.00000, -64.2..."
7,Interfabricas,"POLYGON Z ((-64.09563 -31.45822 0.00000, -64.0..."
8,Sur,"POLYGON Z ((-64.20692 -31.44762 0.00000, -64.2..."
9,Fiat,"POLYGON Z ((-64.13061 -31.44850 0.00000, -64.1..."


In [196]:
df_zs = pd.DataFrame(zonal_stats(SHAPE_ZONAS, lista[10], all_touched=True))

/home/sagus/anaconda3/envs/wrfmeteo/lib/python3.8/site-packages/rasterstats/io.py:301: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


In [197]:
df_zs

,min,max,mean,count
0,24.019480,26.328627,25.349067,9
1,22.999788,25.064100,23.773516,15
2,23.307468,25.243246,23.913520,13
3,24.019480,25.878950,25.064854,4
4,23.256023,24.575445,23.803038,10
5,25.785540,26.570417,26.184123,10
6,24.447849,26.328627,25.551237,12
7,23.077848,24.219383,23.619279,15
8,22.779510,25.243246,23.641807,13
9,23.709459,23.776834,23.743147,2
